<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/tidy_up_preprocessing_notebook/notebooks/processed/ct_preprocessing_jpmorgan_gp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-02-13
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Importing necessary libraries and downloading NLTK data.
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-02-13\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Importing necessary libraries and downloading NLTK data.\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from google.colab import drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# **Function**

preprocessor function : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [3]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


find_row: This function searches upwards from the given current_row_num in the DataFrame df to find the first row where the value in column "M" is "A". It returns the index of that row. If no such row is found, it returns 0

In [4]:
def find_row (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "A":
      break
    else:
      i-=1
  return i

find_row_empty: This function searches upwards from the given current_row_num to find the first row where col1 has the value "A" and col2 is not an empty list. It returns the index of that row.

In [5]:
def find_row_empty (df, col1, col2, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col1][i] == "A" and df[col2][i] != []:
      break
    else:
      i-=1
  return i

In [6]:
def create_ques_num_column (data, new_col,marker_col):
  #Create question number column
  data[new_col]=None
  #set global var to count question number
  num = 0
  #if Q was found, num adds 1 otherwise none
  for i in data.index:
    if data.loc[i,marker_col]=="Q":
      data.at[i,new_col]=num
      num +=1
    else:
      continue

In [7]:
# Function to extract names
def extract_name(full_string):
    return full_string.split(',')[0]

In [8]:
#check if there is "A" before "Q" from the current location
def find_last_a (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "Q":
      j = i-1
      while j > 0:
        if df[col][j]=="A":
          pass
        else:
          break
        j-=1
      break
    else:
      i-=1
  return i

## **Data**

In [9]:
drive.mount('/content/drive')


Mounted at /content/drive


JP Morgan QA section

In [10]:
#Defining qa_data
qa_data = pd.read_excel("/content/drive/MyDrive/bank_of_england/data/preprocessed_data/JP Mogran processed thru OpenAI/JPMorgan_QNA_Consildated_processed_data.xlsx")
qa_data.head()

,Index,Quarter-Year,Question,Asked By,Role of the person asked the question,Answer,Answered By,Role of the person answered the question
0,1,1Q23,"So, Jamie, I was actually hoping to get your p...",Steven Chubak,"Analyst, Wolfe Research LLC","Well, I think you were already kind of complet...",Jamie Dimon,"Chairman & Chief Executive Officer, JPMorgan C..."
1,2,1Q23,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",Ken Usdin,"Analyst, Jefferies LLC","Yeah, sure. So let me just summarize the drive...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co."
2,3,1Q23,"Hi, thanks. Jeremy, wanted to follow up again ...",John McDonald,"Analyst, Autonomous Research","Yeah. John, it's a really good question, and w...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co."
3,4,1Q23,My first question is you mentioned that your r...,Erika Najarian,"Analyst, UBS Securities LLC","Yeah. So, Erika, as you know, we take \n not g...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co."
4,5,1Q23,Hey. Good morning. Maybe just a little bit on ...,Jim Mitchell,"Analyst, Seaport Global Securities LLC","Yeah. A couple things there. So, first of all,...","Jeremy Barnum, Jamie Dimon","Chief Financial Officer, JPMorgan Chase & Co.;..."


In [12]:
#preprocessing data
preprocessor(qa_data, "Question", "question_tokenised_data", "question_cleaned")
preprocessor(qa_data,"Answer","answer_tokenised_data","answer_cleaned")

In [13]:
#remove operater
#qa_data = qa_data.loc[qa_data["speaker"]!="Operator"]

#remove less than 20 words
qa_data["count"] = qa_data["question_tokenised_data"].apply(lambda x: len(x))
qa_data = qa_data.loc[qa_data["count"]>20]
qa_data.head()

,Index,Quarter-Year,Question,Asked By,Role of the person asked the question,Answer,Answered By,Role of the person answered the question,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count
0,1,1Q23,"So, Jamie, I was actually hoping to get your p...",Steven Chubak,"Analyst, Wolfe Research LLC","Well, I think you were already kind of complet...",Jamie Dimon,"Chairman & Chief Executive Officer, JPMorgan C...","[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,54
1,2,1Q23,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",Ken Usdin,"Analyst, Jefferies LLC","Yeah, sure. So let me just summarize the drive...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,35
2,3,1Q23,"Hi, thanks. Jeremy, wanted to follow up again ...",John McDonald,"Analyst, Autonomous Research","Yeah. John, it's a really good question, and w...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[thanks, jeremy, wanted, follow, driver, nii, ...",[hi thanks jeremy wanted follow drivers nii re...,"[yeah, john, really, good, question, obviously...",[yeah john really good question obviously thou...,45
4,5,1Q23,Hey. Good morning. Maybe just a little bit on ...,Jim Mitchell,"Analyst, Seaport Global Securities LLC","Yeah. A couple things there. So, first of all,...","Jeremy Barnum, Jamie Dimon","Chief Financial Officer, JPMorgan Chase & Co.;...","[hey, good, morning, maybe, little, bit, depos...",[hey good morning maybe little bit deposit tho...,"[yeah, couple, thing, there, first, all, know,...",[yeah couple things there so first all know ri...,31
5,6,1Q23,"In your comments about your CET1 ratio, obviou...",Gerard Cassidy,"Analyst, RBC Capital Markets LLC","Yeah. So a few things on there, Gerard. So we ...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[comment, cet, ratio, obviously, came, strong,...",[comments cet1 ratio obviously came strong 138...,"[yeah, thing, there, gerard, previously, said,...",[yeah things there gerard previously said targ...,35


In [14]:
#reset index
qa_data.reset_index(drop=True, inplace=True)
qa_data.head()

,Index,Quarter-Year,Question,Asked By,Role of the person asked the question,Answer,Answered By,Role of the person answered the question,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count
0,1,1Q23,"So, Jamie, I was actually hoping to get your p...",Steven Chubak,"Analyst, Wolfe Research LLC","Well, I think you were already kind of complet...",Jamie Dimon,"Chairman & Chief Executive Officer, JPMorgan C...","[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,54
1,2,1Q23,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",Ken Usdin,"Analyst, Jefferies LLC","Yeah, sure. So let me just summarize the drive...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,35
2,3,1Q23,"Hi, thanks. Jeremy, wanted to follow up again ...",John McDonald,"Analyst, Autonomous Research","Yeah. John, it's a really good question, and w...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[thanks, jeremy, wanted, follow, driver, nii, ...",[hi thanks jeremy wanted follow drivers nii re...,"[yeah, john, really, good, question, obviously...",[yeah john really good question obviously thou...,45
3,5,1Q23,Hey. Good morning. Maybe just a little bit on ...,Jim Mitchell,"Analyst, Seaport Global Securities LLC","Yeah. A couple things there. So, first of all,...","Jeremy Barnum, Jamie Dimon","Chief Financial Officer, JPMorgan Chase & Co.;...","[hey, good, morning, maybe, little, bit, depos...",[hey good morning maybe little bit deposit tho...,"[yeah, couple, thing, there, first, all, know,...",[yeah couple things there so first all know ri...,31
4,6,1Q23,"In your comments about your CET1 ratio, obviou...",Gerard Cassidy,"Analyst, RBC Capital Markets LLC","Yeah. So a few things on there, Gerard. So we ...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[comment, cet, ratio, obviously, came, strong,...",[comments cet1 ratio obviously came strong 138...,"[yeah, thing, there, gerard, previously, said,...",[yeah things there gerard previously said targ...,35


In [ ]:
# Apply the function to the DataFrame
qa_data['speaker'] = qa_data['speaker'].apply(extract_name)

<ipython-input-422-3b2ec04a312b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_data['speaker'] = qa_data['speaker'].apply(extract_name)


In [ ]:
%ls

In [ ]:
#defining santader dataframe
jpmorgan_body_df=pd.read_csv("jpmorgan_management_discussion.csv")
jpmorgan_body_df.head()

In [ ]:
#preprocess data
preprocessor(jpmorgan_body_df, "chunk_text", "tokenized_data","cleaned_data")

In [ ]:
jpmorgan_body_df.head()

UBS qna section

In [ ]:
%ls

In [ ]:
#define ubs q&a data
ubs_qna_df=pd.read_csv("ubs_qna_section.csv")

In [ ]:
#preprocessing ubs Q&A data
preprocessor(ubs_qna_df, "utterance", "tokenized_data","cleaned_data")

In [ ]:
ubs_qna_df.head()

UBS management discussion

In [ ]:
%ls

In [ ]:
#defining ubs management discussion
ubs_manag_df=pd.read_csv("ubs_management_discussion.csv")
ubs_manag_df.head()

In [ ]:
#preprocessing ubs management discussion
preprocessor(ubs_manag_df,"utterance", "tokenized_data","cleaned_data")
ubs_manag_df.head()

# **Export the output as a csv file**

JP morgan QA section

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path1 = "/content/drive/MyDrive/bank_of_england/data/preprocessed_data/jpmorgan_qna_df_preprocessed_ver7.csv"
qa_data.to_csv(preprocessed_qa_csv_path1, index=False)

JP morgan management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path2 = "/content/sample_data/jpmorgan_management_df_preprocessed.csv"
jpmorgan_body_df.to_csv(preprocessed_qa_csv_path2, index=False)

UBS QA section

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path3 = "/content/sample_data/ubs_qa_df_preprocessed.csv"
ubs_qna_df.to_csv(preprocessed_qa_csv_path3, index=False)

UBS management discussion

In [ ]:
#export preprocessed data
preprocessed_qa_csv_path4 = "/content/sample_data/ubs_management_df_preprocessed.csv"
ubs_manag_df.to_csv(preprocessed_qa_csv_path4, index=False)